In [1]:
import numpy as np 
import pandas as pd 
import torch # for models
from torch import nn
import torchvision
from torchvision import transforms, datasets
import pathlib
from torchvision.datasets import ImageFolder
import torch.nn.functional as F
import torchmetrics
from torch.utils.data import DataLoader, SubsetRandomSampler, Subset
from sklearn.model_selection import train_test_split

In [2]:
torch.cuda.is_available()
torch.cuda.current_device()

0

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
class CNN(nn.Module):
    def __init__(self,number_of_classes):
        super().__init__() #Inheritance
        
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=16,padding=1,kernel_size=3)
        self.bn1=nn.BatchNorm2d(num_features=16)
        self.relu1=nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(in_channels=16,out_channels=32,padding=1,kernel_size=3)
        self.bn2=nn.BatchNorm2d(num_features=32)
        self.relu2=nn.ReLU()
        
        self.conv3 = nn.Conv2d(in_channels=32,out_channels=64,padding=1,kernel_size=3)
        self.bn3=nn.BatchNorm2d(num_features=64)
        self.relu3=nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.fc=nn.Linear(64*56*56, number_of_classes)

    
    def forward(self, Input):
            
        output=self.conv1(Input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool1(output)
        
        output=self.conv2(output)
        output=self.bn2(output)
        output=self.relu2(output)
        
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        output=self.pool3(output)
        
        output = torch.flatten(output, 1)
        output = self.fc(output)
        
        return output

In [5]:
# preprocessing
preprocess = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

path = pathlib.Path("data/Rice_Image_Dataset")
dataset = datasets.ImageFolder(path, transform=preprocess)

# dataset loader
BATCH_SIZE = 256

# Number of classes
NUM_CLASSES = 5

# train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, (0.7, 0.2, 0.1))
# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
# Get the target values (labels) from the dataset
targets = np.array(dataset.targets)

val_prop = 0.2
test_prop = 0.1

# Split the dataset into train and test sets
train_val_indices, test_indices, train_val_targets, test_targets = train_test_split(np.arange(len(dataset)), targets, test_size=test_prop, stratify=targets)

# Split the train set into train and validation sets
train_indices, val_indices, train_targets, val_targets = train_test_split(train_val_indices, train_val_targets, test_size=val_prop, stratify=train_val_targets)

# Create custom PyTorch datasets for the train, validation, and test sets using the original dataset and the indices of the split data
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

# Create custom dataloaders for the train, validation, and test sets
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
def train_model(model, train_dataloader=train_dataloader, val_dataloader=val_dataloader, device=device, lr=1e-3, momentum=0.9, num_classes=5, epochs=1):
    # Freeze the weights of the model
    for param in model.parameters():
        param.requires_grad = False
    model_in_features = model.fc.in_features
    model.fc = torch.nn.Linear(model_in_features, num_classes).to(device)
    optimizer = torch.optim.SGD([
                        {'params': model.fc.parameters()}
                    ],
                    lr=lr,
                    momentum=momentum
                )
    model.train()
    for epoch in range(epochs):
        print(f'running epoch {epoch+1}')
        train_loss = 0.0
        train_acc = 0.0
        for inputs, targets in train_dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            # Zero the gradients
            optimizer.zero_grad()
            # Forward pass
            outputs = model(inputs)
            loss = torch.nn.functional.cross_entropy(outputs, targets)
            train_loss += loss.item() * inputs.size(0)
            _, predictions = torch.max(outputs, 1)
            train_acc += torch.sum(predictions == targets.data)
            # Backward pass
            loss.backward()
            optimizer.step()

        train_loss /= len(train_dataloader.dataset)
        train_acc /= len(train_dataloader.dataset)
        val_loss, val_acc = get_acc(model=model, dataloader=val_dataloader, num_classes=num_classes)
        print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}, Val Loss: {:.4f}, Val Acc: {:.4f}'
              .format(epoch+1, epochs, train_loss, train_acc, val_loss, val_acc))

    return model

def get_model_specs(model):
    total_params = 0 #default value
    total_params = sum(
        param.numel() for param in model.parameters()
    )
    return total_params

def evaluate_model(model, train_dataloader, val_dataloader, test_dataloader, num_classes=5):
    print('collecting param count')
    total_params = get_model_specs(model)
    print('collecting train accuracy')
    train_acc, train_loss = get_acc(model=model, dataloader=train_dataloader, num_classes=num_classes)
    print('collecting validation accuracy')
    val_acc, val_loss = get_acc(model=model, dataloader=val_dataloader, num_classes=num_classes)
    print('collecting test accuracy')
    test_acc, test_loss = get_acc(model=model, dataloader=test_dataloader, num_classes=num_classes)
    metrics_dict = {
        'total_params': total_params,
        'train_acc': train_acc,
        'train_loss': train_loss,
        'val_acc': val_acc,
        'val_loss': val_loss,
        'test_acc': test_acc,
        'test_loss': test_loss
    }
    metrics_idx = list(metrics_dict.keys())
    metrics = pd.Series(data=metrics_dict, index=metrics_idx)
    return metrics

def get_acc(model, dataloader, num_classes):
    model.eval()
    predictions = []
    targets = []
    total_loss = 0.0
    total_samples = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            # Move the inputs and labels to the device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            preds = F.softmax(outputs, dim=1)

            # Calculate the loss
            loss = torch.nn.functional.cross_entropy(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            total_samples += inputs.size(0)

            # Store the predictions and targets
            predictions.extend(preds.cpu().detach().numpy())
            targets.extend(labels.cpu().detach().numpy())

    # Calculate the accuracy and average loss
    accuracy = torchmetrics.functional.accuracy(torch.tensor(predictions), torch.tensor(targets), num_classes=num_classes, task='multiclass')
    avg_loss = total_loss / total_samples
    
    return accuracy, avg_loss

In [8]:
# model loading
# resnet_model = torchvision.models.resnet34(pretrained=True).to(device)
# alexnet_model = torchvision.models.alexnet(pretrained=True).to(device)
# vgg_model = torchvision.models.vgg16(pretrained=True).to(device)
# resnet_model_trained = train_model(resnet_model, train_dataloader=train_dataloader, device=device, num_classes=NUM_CLASSES, epochs=8)
# metrics = evaluate_model(resnet_model_trained, train_dataloader, val_dataloader, test_dataloader, 5)

In [9]:
# train CNN
cnn_model = CNN(5).to(device)
cnn_model_trained = train_model(cnn_model, train_dataloader=train_dataloader, device=device, num_classes=NUM_CLASSES, epochs=8)
metrics_cnn = evaluate_model(cnn_model_trained, train_dataloader, val_dataloader, test_dataloader, 5)
metrics_cnn

running epoch 1


UnboundLocalError: local variable 'total_loss' referenced before assignment